In [16]:
import os
import re

import cv2
import dlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from cv2.data import haarcascades
from keras.models import load_model
from tqdm import tqdm

In [3]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(data, key=alphanum_key)

In [ ]:
print("Loading model FaceQnet_v1 for scoring faces ...")
model = load_model(os.path.join("models", "FaceQnet_v1.h5"))
print("Model loaded!")

# CV2 | Haar Cascade method

## Multiple images (Main algorithm)

In [ ]:
path = os.path.join("IMGs", "fake","Face2Face")
output = os.path.join("output", "fake", "Face2Face", "cropped_faces")

if not (os.path.exists(output)):
    os.makedirs(output)

output_size = (384, 384)
checkpoint = '' # Example: 409_382-frame57.jpg

face_cascade = cv2.CascadeClassifier(haarcascades +'haarcascade_frontalface_alt.xml')

with open('multiple_faces_fake_face2face.txt', 'a') as file:

    print("Sorting files ...")
    lp = sorted_alphanumeric(os.listdir(path))
    listPictures = lp[lp.index(checkpoint):]
    print("Sort done! Starting detection... ")

    # Loop over the images
    for image_file in tqdm(listPictures):
        i = 0
        face_list = []
        dim_face_list = []

        # Load image
        img = (cv2.imread(os.path.join(path, image_file), cv2.IMREAD_COLOR))[:, :, ::-1]
        
        # read the haarcascade to detect the faces in an image
        faces = face_cascade.detectMultiScale(img, 1.1, 4)

        # Loop over all detected faces
        if len(faces) == 1:
            for i, (x, y, w, h) in enumerate(faces):
                # To draw a rectangle in a face
                face = img[y:y+h, x:x+w]
        elif len(faces) > 1:
            file.write(image_file + '\n')
            
            for i, (x,y,w,h) in enumerate(faces):
                face = img[y:y+h, x:x+w]
                face_list.append(face)
                height, width, channel = face.shape
                
                resized_face = np.array(cv2.resize(face, (224, 224)), copy=False, dtype=np.float32)
                score = model.predict(np.reshape(resized_face, (1, 224, 224, 3)), batch_size=1, verbose=0)
                
                # Takes only the height because the image are nxn
                dim_face_list.append([height, score, face])

            max_height_index = max(range(len(dim_face_list)), key=lambda i: dim_face_list[i][0])
            max_score_index = max(range(len(dim_face_list)), key=lambda i: dim_face_list[i][1])

            if max_height_index == max_score_index:
                max_row = dim_face_list[max_height_index]
            else:
                max_row = max(dim_face_list, key=lambda row: row[0])

            face = max_row[2]

        else:
            file.write(image_file + ' | No face detected\n')
            continue

        cv2.imwrite(os.path.join(output, image_file), cv2.resize(face[:, :, ::-1], output_size))

## Single image

In [ ]:
image = '' #337-frame72.jpg
switchRGB = True
#index = 1

path = os.path.join("IMGs", "real", image)
output = os.path.join("output", "real", "cropped_faces")
output_size = (384, 384)

face_cascade = cv2.CascadeClassifier(haarcascades +'haarcascade_frontalface_alt.xml')               
                
scores = []
face = None

# Load image
img = (cv2.imread(os.path.join(path), cv2.IMREAD_COLOR))

if(switchRGB):
    img = img[:, :, ::-1]

# Read the haarcascade to detect the faces in an image
faces = face_cascade.detectMultiScale(img, 1.1, 10)

if len(faces) < 1:
    print('No face detected for this image.')
elif len(faces) == 1:
    for i, (x, y, w, h) in enumerate(faces):
        # To draw a rectangle in a face
        face = img[y:y+h, x:x+w]
else:
    for i, (x, y, w, h) in enumerate(faces):
        face = img[y:y+h, x:x+w]
        print(face.shape)

        resized_face = np.array(cv2.resize(face, (224, 224)), copy=False, dtype=np.float32)
        score = model.predict(np.reshape(resized_face, (1, 224, 224, 3)), batch_size=1, verbose=0)
        print(score)
        scores.append([score, face])

    max_score = max(scores, key=lambda x: x[0])
    #max_score = scores[index]
    face = max_score[1]

if face is not None:

    if(switchRGB):
        face = face[:, :, ::-1]

    plt.imshow(face)
    cv2.imwrite(os.path.join(output, image), cv2.resize(face, output_size))

# Dlib

## V1 (dlib) with controls on cropping

In [ ]:
def controlFaceCrop(l, t, r, b, img, fixSize):
        if l < 0:
            l = 0
        if r > img.shape[1]:
            r = img.shape[1]
        if t < 0:
            t = 0
        if b > img.shape[0]:
            b = img.shape[0]

        # Calculate width and height of bounding box
        width = r - l
        height = b - t

        # Determine the scaling factor to resize the bounding box to the fixed size
        scale = max(width, height) / max(fixSize)

        # Calculate the new dimensions for the bounding box
        new_right = int(round(width / scale))
        new_bottom = int(round(height / scale))

        # Calculate the new top-left corner coordinates for the bounding box
        new_left = int(round(l + (width - new_right) / 2))
        new_top = int(round(t + (height - new_bottom) / 2))

        if new_left < 0:
            new_left = 0
        if new_right > img.shape[1]:
            new_right = img.shape[1]
        if new_top < 0:
            new_top = 0
        if new_bottom > img.shape[0]:
            new_bottom = img.shape[0]

        # Extract face crop using bounding box
        return img[new_top:new_top+new_right, new_left:new_left+new_bottom, :]

In [ ]:
# Load face detection model
detector = dlib.get_frontal_face_detector()

path = os.path.join("IMGs", "real")

fixed_size = (384, 384)

checkpoint = '000-frame0.jpg'

with open('multiple_faces.txt', 'a') as file:

    print("Sorting files ...")
    lp = sorted_alphanumeric(os.listdir(path))
    listPictures = lp[lp.index(checkpoint):]
    print("Sort done! Starting detection... ")

    # Loop over the images
    for image_file in tqdm(listPictures):
        scores = []

        # Load image
        img = cv2.imread(os.path.join(path, image_file), cv2.IMREAD_COLOR)

        # Detect faces
        faces = detector(img)
        
        if(len(faces) < 1):
            # No faces here
            file.write(image_file + ' | No face detected\n')
            continue
        elif len(faces) == 1:
            for face in faces:
                left = face.left()
                top = face.top()
                right = face.right()
                bottom = face.bottom()

                face_crop = controlFaceCrop(left, top, right, bottom, img)
                
                cv2.imwrite(os.path.join("output", "real", "cropped_faces", image_file),
                            cv2.resize(np.array(face_crop, copy=False, dtype=np.float32), fixed_size))        
        else:
            file.write(image_file + '\n')

            # Loop over each face detected
            for face in faces:
                
                left = face.left()
                top = face.top()
                right = face.right()
                bottom = face.bottom()

                # Extract face crop using bounding box
                face_crop = controlFaceCrop(left, top, right, bottom, img)

                # Resize face crop to (3, 224, 224)            
                resized_face = np.array(cv2.resize(face_crop, (224, 224)), copy=False, dtype=np.float32)
                score = model.predict(np.reshape(resized_face, (1, 224, 224, 3)), batch_size=1, verbose=0)
                scores.append([score, image_file, resized_face])

            max_score = max(scores, key=lambda x: x[0])
            cv2.imwrite(os.path.join("output", "real", "cropped_faces", max_score[1]), cv2.resize(max_score[2], fixed_size))

## V1 alt. (dlib) without controls on cropping

In [ ]:
# Load face detection model
detector = dlib.get_frontal_face_detector()

path = os.path.join("IMGs", "real")

fixed_size = (384, 384)

checkpoint = '000-frame0.jpg'

with open('multiple_faces.txt', 'a') as file:

    print("Sorting files ...")
    lp = sorted_alphanumeric(os.listdir(path))
    listPictures = lp[lp.index(checkpoint):]
    print("Sort done! Starting detection... ")

    # Loop over the images
    for image_file in tqdm(listPictures):
        scores = []

        # Load image
        img = cv2.imread(os.path.join(path, image_file), cv2.IMREAD_COLOR)

        # Detect faces
        faces = detector(img)
        
        if(len(faces) < 1):
            # No faces here
            file.write(image_file + ' | No face detected\n')
            continue
        elif len(faces) == 1:
            for face in faces:
                left = face.left()
                top = face.top()
                right = face.right()
                bottom = face.bottom()

                # Extract face crop using bounding box
                face_crop = img[top:top+bottom, left:left+right, :]
                
                cv2.imwrite(os.path.join("output", "real", "cropped_faces", image_file),
                            cv2.resize(np.array(face_crop, copy=False, dtype=np.float32), fixed_size))        
        else:
            file.write(image_file + '\n')

            # Loop over each face detected
            for face in faces:
                
                left = face.left()
                top = face.top()
                right = face.right()
                bottom = face.bottom()

                # Extract face crop using bounding box
                face_crop = img[top:top+bottom, left:left+right, :]

                # Resize face crop to (3, 224, 224)            
                resized_face = np.array(cv2.resize(face_crop, (224, 224)), copy=False, dtype=np.float32)
                score = model.predict(np.reshape(resized_face, (1, 224, 224, 3)), batch_size=1, verbose=0)
                scores.append([score, image_file, resized_face])

            max_score = max(scores, key=lambda x: x[0])
            cv2.imwrite(os.path.join("output", "real", "cropped_faces", max_score[1]), cv2.resize(max_score[2], fixed_size))

# Reorder the file about multiple faces

In [15]:
# Use real_images = True if you are working with images from the "Real" class.
# Use real_images = False if you are working with images from the "Fake" class.
real_images = False
path = 'multiple_faces_fake_face2face.txt'
output = 'final_fake_face2face.txt'

# Read the file into a pandas dataframe
df = pd.read_csv(path, header=None, names=['string'])

if(real_images):
    # Extract the video and frame numbers into separate columns
    df[['video', 'frame']] = df['string'].str.extract(r'(\d+)-frame(\d+)\.jpg')
else:
     # Extract the video and frame numbers into separate columns
    df[['video', 'fakeVideo', 'frame']] = df['string'].str.extract(r'(\d+)_(\d+)-frame(\d+)\.jpg')

# Convert the video and frame columns to integers
df['video'] = df['video'].astype(str)
df['frame'] = df['frame'].astype(str)

# Group the data by 'video'
grouped = df.groupby('video')

# Create a list to store the lines for the output file
lines = []
i = 0

# Iterate over each group
for video, group in grouped:
    frames = []

    if not real_images:
        video = video.split("_")[0]
        
    lines.append(f'Video: {video} (Possible multiple faces)')

    # Check if any frames have "No face detected" message
    no_face_frames = group[group['string'].str.contains('No face detected')]

    # Add lines for frames with "No face detected"
    if not no_face_frames.empty:
        for _, row in no_face_frames.iterrows():
            frames.append(str(row['frame']))
            i += 1

        result = '-'.join(frames)
        lines.append(f'Video: {video} | No face detected at frame {result}.')

    lines.append("======================================")

# Write the lines to the output file
with open(output, 'w') as f:
    f.write('\n'.join(lines))

print(F"No faces detected for {i} frames.")

No faces detected for 428 frames.
